详细配置参考：
- https://wiki.archlinux.org/index.php/Hardware_video_acceleration_(%E7%AE%80%E4%BD%93%E4%B8%AD%E6%96%87)#VA-API
- https://wiki.archlinux.org/index.php/Hardware_video_acceleration#VA-API_drivers

- https://blogs.igalia.com/vjaquez/2018/03/28/gstreamer-va-api-troubleshooting/


## 查看当前显卡信息

查看当前使用的显卡驱动

```sh
lspci -vnn | grep VGA -A 12
lshw -C display
sudo lshw -c video | grep configuration

lspci |grep VGA 

```
信息： 
```
01:00.0 VGA compatible controller: NVIDIA Corporation GK106 [GeForce GTX 645 OEM] (rev a1)
```

解析：  
GK106，是一个由英伟达公司制造的适配器图形处理和加速核心芯片。

从NVIDIA支持的矩阵表中查找出GK106支持格式分别为：   
NVDEC解码：MPEG-2 VC-1 H.264(AVCHD)   
NVENC编码：H.264 (AVCHD) YUV 4:2:0

- [NVIDIA支持的矩阵表](https://developer.nvidia.com/video-encode-decode-gpu-support-matrix)
- [Linux中如何查看显卡硬件信息](https://www.linuxidc.com/Linux/2015-12/126627.htm)

# Nvidia gstreamer硬件加速

## 安装基础包

```sh
sudo apt-get install autoconf automake libtool build-essential  libass-dev pkg-config texinfo zlib1g-dev cmake mercurial libvorbis-dev libogg-dev git git-core libperl-dev libpciaccess-dev libpciaccess0 libfreetype6-dev libsdl2-dev libxcb1-dev libxcb-shm0-dev libxcb-xfixes0-dev wget zlib1g-dev    
```

## 安装VA-API（libav)相关库

```sh
sudo apt-get install libva libva-dev libva-x11-1 libx11-dev  libdrm-dev libva-glx1 libudev-dev mesa-common-dev xorg-dev vainfo
```

```sh
sudo apt-get install libva libva-dev libx11-dev  libdrm-dev libudev-dev mesa-common-dev xorg-dev vainfo
```



## 安装VDPAU

```sh
sudo apt-get install libvdpau-dev libvdpau1 vdpauinfo
```

## 安装VDPAU后端驱动

1. 方式一：直接安装

```sh
sudo apt install vdpau-va-driver
```

2. 方式二：编译安装最新版

  驱动自带的`vdpau-va-driver`包有些bug
`vdpau_EndPicture`: 在vdpau_endpicture函数中产生异常

```sh
git clone git://anongit.freedesktop.org/vaapi/vdpau-driver
cd vdpau-driver/
./autogen.sh --prefix=/usr
make
```
  移除引起安装问题的链接：
  
```sh
cd /usr/lib/arm-linux-gnueabihf/dri
rm -f nvidia_drv_video.so s3g_drv_video.so
cd -
```

  安装驱动器：
```sh
make install
```

## 确定VA-API系统正常工作

详细配置参考：[Configuring_VDPAU](https://wiki.archlinux.org/index.php/Hardware_video_acceleration#Configuring_VDPAU)

#在~/.bashrc文件中添加下面三行(永久)

```sh
export VDPAU_DRIVER=nvidia
export LIBVA_DRIVER_NAME=vdpau
export LIBVA_DRIVERS_PATH=/usr/lib/x86_64-linux-gnu/dri
```
查看系统信息：
```sh
vdpauinfo
vainfo
```

正常情况下应该会有类似这样的输出：
```sh
libva: VA-API version 0.32.0
libva: User requested driver 'vdpau'
libva: Trying to open /usr/lib/arm-linux-gnueabihf/dri/vdpau_drv_video.so
libva: va_openDriver() returns 0
[VDPAU SUNXI] VE version 0x1623 opened.
vainfo: VA-API version: 0.32 (libva 1.0.15)
vainfo: Driver version: Splitted-Desktop Systems VDPAU backend for VA-API - 0.7.5.pre1
vainfo: Supported profile and entrypoints
      VAProfileMPEG2Simple            : VAEntrypointVLD
      VAProfileMPEG2Main              : VAEntrypointVLD
      VAProfileMPEG4Simple            : VAEntrypointVLD
      VAProfileMPEG4AdvancedSimple    : VAEntrypointVLD
      VAProfileH264Baseline           : VAEntrypointVLD
      VAProfileH264Main               : VAEntrypointVLD
      VAProfileH264High               : VAEntrypointVLD
```      

`VAEntrypointVLD` 意味着你可以解码这个格式  
`VAEntrypointEncSlice`意味着你可以编码这个格式
参考：

- [香蕉派获取硬件加速](http://forum.lemaker.org/cn/thread-2237-1-1-%E3%80%90%E8%BD%AC%E3%80%91%E5%A6%82%E4%BD%95%E5%88%A9%E7%94%A8gstreamer%E6%88%90%E5%8A%9F%E8%8E%B7%E5%BE%97%E7%A1%AC%E4%BB%B6%E5%8A%A0%E9%80%9F%E8%A7%86%E9%A2%91.html)

# Intel 硬件加速

## 安装基础包

```sh
sudo apt-get install autoconf automake libtool build-essential  libass-dev pkg-config texinfo zlib1g-dev cmake mercurial libvorbis-dev libogg-dev git git-core libperl-dev libpciaccess-dev libpciaccess0 libfreetype6-dev libsdl2-dev libxcb1-dev libxcb-shm0-dev libxcb-xfixes0-dev wget zlib1g-dev    
```

## 安装VAAPI（libav)相关库

```sh
sudo apt-get install libdrm-intel1 libva1 libva-intel-vaapi-driver libva-x11-1 libx11-dev libva-dev libdrm-dev libudev-dev mesa-common-dev xorg-dev vainfo
```
 vainfo
源码安装Libav：

```sh
git clone https://github.com/intel/libva --depth=1
cd libva
./autogen.sh
./configure
make
sudo make install
```

## 安装VAAPI后端驱动

安装Intel集显驱动：

方式一：在线源
```sh
sudo apt-get install i965-va-driver
```
方式二：编译安装

**intel-vaapi-driver：**

```sh
git clone https://github.com/intel/intel-vaapi-driver --depth=1
cd intel-vaapi-driver
./autogen.sh
./configure
make
sudo make install
```

## 检查

首先查看是否有intel驱动：
```sh
apt list i965-va-driver
#运行之后，有驱动说明成功安装
```

#在~/.bashrc文件中添加下面两行(永久)

```sh
export LIBVA_DRIVER_NAME=i965 
export LIBVA_DRIVERS_PATH=/usr/lib/x86_64-linux-gnu/dri
```

查看libva版本
```sh 
dpkg-query --showformat='${Package}: ${Version}\n' --show | grep libva
```
查看vainfo版本
```sh
dpkg-query --showformat='${Package}: ${Version}\n' --show | grep vainfo
```
查看i965版本
```sh
dpkg-query --showformat='${Package}: ${Version}\n' --show | grep i965
```

主要是验证：
```sh
vainfo
```

其他安装完成之后，若还是运行vainfo没有显示
```
vainfo: Supported profile and entrypoints
      VAProfileMPEG2Simple            :    VAEntrypointVLD
      VAProfileMPEG2Simple            :    VAEntrypointEncSlice
      VAProfileMPEG2Main              :    VAEntrypointVLD
      VAProfileMPEG2Main              :    VAEntrypointEncSlice
      VAProfileH264ConstrainedBaseline:    VAEntrypointVLD
      VAProfileH264ConstrainedBaseline:    VAEntrypointEncSlice
      VAProfileH264ConstrainedBaseline:    VAEntrypointEncSliceLP
      VAProfileH264Main               :    VAEntrypointVLD
      VAProfileH264Main               :    VAEntrypointEncSlice
      VAProfileH264Main               :    VAEntrypointEncSliceLP
      VAProfileH264High               :    VAEntrypointVLD
```
则需要以下操作：
```
vainfo --display drm --device /dev/dri/renderD128
或者
vainfo --display drm --device /dev/dri/renderD129
```

分别运行看那行代码有显示上面的内容。

最后需要操作：

```sh
$ gedit ~/.bash_aliases
 
alias vainfo='vainfo --display drm --device /dev/dri/renderD128'（或者129，看那个能用）
#这个时候就可以直接使用vainfo
```

      
## 使用gstreamer

### 安装gstreamer包

```sh
apt-get install libgstreamer1.0-0 gstreamer1.0-plugins-base gstreamer1.0-plugins-good gstreamer1.0-plugins-bad gstreamer1.0-plugins-ugly gstreamer1.0-libav gstreamer1.0-doc gstreamer1.0-tools gstreamer1.0-x gstreamer1.0-alsa gstreamer1.0-gl gstreamer1.0-gtk3 gstreamer1.0-qt5 gstreamer1.0-pulseaudio
```

### 安装gstreamer VAAPI的依赖包

```sh
apt-get install libglib2.0-dev libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev libgstreamer-plugins-bad1.0-dev mesa-vdpau-drivers
```

### 安装gstreamer VAAPI 插件

- 方式一：

```sh
sudo apt install gstreamer1.0-vaapi
```

- 方式二：编译安装

下载地址: [gstreamer-vaapi/](https://gstreamer.freedesktop.org/src/gstreamer-vaapi/)

**注意**： 下载解压后检查 README 文件，里面写明了的依赖包以及版本环境，如果不支持当前环境，请重新下载新的或者旧的版本。（也可从configure脚本文件里查找 Gstreamer API定位到相关代码查看gstreamer 的版本要求。）

解压 xz 文件：

```sh
xz -d gstreamer-vaapi-xxx.tar.xz
```
生成 tar 文件，再次解压：

```sh
tar -xvf gstreamer-vaapi-xxx.tar
```
编译安装：

```sh
cd gstreamer-vaapi/
./autogen.sh --prefix=/usr
make
make install
cd ..
```

### 检查是否成功

```sh
gst-inspect-1.0 | grep -i vaapi
```
正常情况下应该会有类似这样的输出：
```
vaapiparse:  vaapiparse_h264: H.264 parser
vaapi:  vaapidecode: VA-API decoder
vaapi:  vaapipostproc: VA-API video postprocessing
vaapi:  vaapisink: VA-API sink
```

# AMD 硬件加速

## 安装基础包

## 安装XVBA后端驱动

```sh
sudo apt-get install xvba-va-driver
```


# 说明

- **libva**：也就是传说中的 VAAPI，是一个开放源代码的提供硬件解码功能的模块；  
- **i965-va-driver**：Intel 显卡的 VAAPI 后端，负责连接 Intel 显示卡驱动与 VAAPI。  
- **xvba-video**：AMD/ATi 显卡的 VAAPI 后端，负责连接 AMD/ATi 显示卡驱动与 VAAPI。  
- **vdpau**：NV 显卡的 VAAPI 后端, 负责连接 NV 显示卡驱动与 VAAPI。

# 参考

- [Intel VAAPI 详细安装](https://01.org/linuxmedia/documentation/source-code)

- [gstreamer安装AV-VPI成功后没有查询不到vaapi插件问题解决](https://blog.csdn.net/DrBlack666/article/details/102669611)

- [install mplayer with va-api hardware acceleration for intel amd gpus support in ubuntu](https://www.tuicool.com/articles/EFbU3m)